In [1]:
import os
import sys
import django

sys.path.append(
os.path.dirname(os.getcwd()))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "advance_django.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"
django.setup()

In [ ]:
from django.db import models


class Author(models.Model):
    name = models.CharField(max_length=100)


class Book(models.Model):
    title = models.CharField(max_length=100)
    author = models.ForeignKey(Author, on_delete=models.CASCADE)


class Review(models.Model):
    book = models.ForeignKey(Book, on_delete=models.CASCADE)
    rating = models.IntegerField()  # rating out of 5
    comment = models.TextField()

## Imports

In [5]:
from orm.models import Author, Book, Review
from django.db.models import F
from django.db.models.functions import Upper

## Data Population

In [ ]:

a1 = Author.objects.create(name="User 1")
a2 = Author.objects.create(name="User 2")

b1 = Book.objects.create(author=a1, title="Hello World")
b2 = Book.objects.create(author=a2, title="Nemesis")

r1 = Review.objects.create(book=b1, rating=3, comment="Nice One")
r2 = Review.objects.create(book=b2, rating=5, comment="Great One")


## alias

### aliased field cann't be used in values.We need annotate for that

In [ ]:
qs = (
    Author.objects.alias(author_name_upper_case=Upper("name"))
    .filter(author_name_upper_case="USER 1")
    .values("name")
)
print(qs)

<QuerySet [{'name': 'User 1'}]>


## extra

In [ ]:
qs = Author.objects.extra(
    select={"author_name_upper_case": "SELECT UPPER(name)"}
).values("author_name_upper_case")
print(qs)

<QuerySet [{'name': 'User 1'}, {'name': 'User 2'}]>


## select_for_update

In [16]:
qs = Author.objects.select_for_update().values("name")
print(qs)

<QuerySet [{'name': 'User 1'}, {'name': 'User 2'}]>


## defer and only

### only pick those columns which you need instead of bulkly all result

In [ ]:
a1 = Review.objects.defer("comment").first()
a2 = Review.objects.only("rating").last()

print(a1.rating)
print(a2.rating)

# RESULT IN NEW SQL QUERIES
print(a1.comment)
print(a2.comment)

3
5
Nice One
Great One


## in_bulk

In [20]:
result = Author.objects.in_bulk([1, 2])
print(result)

{1: <Author: Author object (1)>, 2: <Author: Author object (2)>}


## iterator

In [21]:
result = Author.objects.iterator()

for author in result:
    print(author.name)

User 1
User 2


## latest() and earliest()

In [ ]:
# Assuming we have a 'published_date' field
latest_post = Book.objects.latest('published_date')
earliest_post = Book.objects.earliest('published_date')

## exists()

In [ ]:
if Author.objects.filter(name__icontains="1").exists():
    print("Such Author Exits")

Such Author Exits


In [32]:
qs = Review.objects.all()
qs.explain()

'2 0 0 SCAN orm_review'